In [2]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [3]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [5]:
SELECT NomeMedicamento, COUNT(*) AS qtdPessoas
    FROM UsoMedicamento, Medicamento
    WHERE Codigo = CodMedicamento
    GROUP BY NomeMedicamento

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [40]:
DROP VIEW IF EXISTS UsosPorMedicamento;
CREATE VIEW UsosPorMedicamento AS
SELECT CodMedicamento as cm, COUNT(IdPessoa) AS qtdPessoas
    FROM UsoMedicamento
    GROUP BY CodMedicamento

In [42]:
SELECT cm  
    FROM UsosPorMedicamento 
    WHERE qtdPessoas = (SELECT MAX(qtdPessoas) FROM UsosPorMedicamento)

d00732

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [19]:
SELECT CodMedicamento
    FROM UsoMedicamento
    GROUP BY CodMedicamento
    HAVING COUNT(IdPessoa) = (SELECT MAX(quantidade) FROM (SELECT COUNT(IdPessoa) AS quantidade FROM UsoMedicamento GROUP BY CodMedicamento))

d00732

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [20]:
SELECT NomeMedicamento
    FROM UsoMedicamento, Medicamento
    WHERE Codigo = CodMedicamento
    GROUP BY NomeMedicamento
    HAVING COUNT(IdPessoa) = (SELECT MAX(quantidade) FROM (SELECT COUNT(IdPessoa) AS quantidade FROM UsoMedicamento GROUP BY CodMedicamento))

LISINOPRIL

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [31]:
SELECT AVG(quantidade) FROM (SELECT COUNT(CodMedicamento) AS quantidade FROM UsoMedicamento GROUP BY IdPessoa)

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [34]:
SELECT IdPessoa, COUNT(CodMedicamento) qtdMedicamentos
    FROM UsoMedicamento
    GROUP BY IdPessoa
    HAVING qtdMedicamentos > (SELECT AVG(quantidade) FROM (SELECT COUNT(CodMedicamento) AS quantidade FROM UsoMedicamento GROUP BY IdPessoa))
    ORDER BY qtdMedicamentos

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [44]:
DROP VIEW IF EXISTS UsosPorPessoa;
CREATE VIEW UsosPorPessoa AS
SELECT IdPessoa, COUNT(CodMedicamento) qtdMedicamentos
    FROM UsoMedicamento
    GROUP BY IdPessoa

In [47]:
SELECT UsosPorPessoa.IdPessoa, Pessoa.Idade, UsosPorPessoa.qtdMedicamentos
    FROM UsosPorPessoa, Pessoa
    WHERE UsosPorPessoa.IdPessoa = Pessoa.Id
    ORDER BY Pessoa.Idade